# Market Data Integration Guide

## Overview
This guide covers integrating with TastyTrade's market data API using Python WebSocket implementation, with a focus on asyncio-based approach.

## TastyTrade API Resources
- [Developer Portal](https://developer.tastytrade.com/)
- [Streaming Market Data Documentation](https://developer.tastytrade.com/streaming-market-data/)
- [DXLink Protocol Reference](https://demo.dxfeed.com/dxlink-ws/debug/#/protocol)

## WebSocket Implementation Details

### Selected Library
We'll be using the [websockets](https://websockets.readthedocs.io/en/stable/) library for Python, which provides high-level WebSocket implementation (not to be confused with the lower-level `websocket` library).

### Key Resources
- **Documentation**: Complete guides available on [readthedocs.io](https://websockets.readthedocs.io/en/stable/)
- **Reference Implementation**: [Example using Threading](https://github.com/LordKaT/tastytrade_api_thing/blob/main/lib/TTWebsocket.py) by [LordKaT](https://github.com/LordKaT)

### Technical Approach
The implementation will use `asyncio` due to its comprehensive [feature support](https://websockets.readthedocs.io/en/stable/reference/features.html) in the websockets library.

### AsyncIO Learning Resources
For better understanding of the asyncio implementation:
- [How does async/await work in Python 3.5](https://snarky.ca/how-the-heck-does-async-await-work-in-python-3-5/)
- [Cooperative multitasking with Coroutines](https://pymotw.com/3/asyncio/coroutines.html)
- [A Curious course on Coroutines and Concurrency](http://www.dabeaz.com/coroutines/)

## Implementation Notes
While previous experience has been primarily with traditional threading, this project will explore asyncio-based implementation for its potential benefits in handling WebSocket connections.

In [1]:
import logging

import asyncio
from tastytrade.logging import setup_logging
from tastytrade.sessions import Credentials
from tastytrade.sessions.requests import AsyncSessionHandler
from tastytrade.sessions.sockets import WebSocketManager

from tastytrade.sessions.dxlink import DXLinkClient

logging.getLogger().handlers.clear()
logger = logging.getLogger(__name__)
setup_logging(logging.DEBUG)

TEST = True
ENV = "Live"

shutdown = asyncio.Event()

In [ ]:
session = await AsyncSessionHandler.create(Credentials(env=ENV))

async with WebSocketManager(session) as websocket:
    dxlink_client = DXLinkClient(websocket)

    # await dxlink_client.add_feed()
    # await dxlink_client.subscribe_to_feed(channel)

    # Needed to keep the websocket open
    # await websocket.send_keepalives()
    await asyncio.sleep(5)

await session.close()

In [ ]:
session = await AsyncSessionHandler.create(Credentials(env=ENV))

exlink = WebSocketManager(session)
txlink = WebSocketManager(session)
print(id(exlink) == id(txlink))

In [ ]:
txlink.__dict__

In [ ]:
exlink.sessions

In [ ]:
await exlink.open()

In [4]:
dxlink_client = DXLinkClient(exlink)

In [ ]:
await exlink.close()

In [ ]:
exlink.sessions

In [9]:
# Injector Module Example

# def configure(binder):
#     binder.bind(Credentials, to=Credentials(env=ENV))


# injector = Injector([configure])
# session = injector.get(SessionHandler)

# session.get_api_quote_token()

# session = SessionHandler.create(Credentials(env="Test"))

# if TEST:
#     session.create_session(Credentials(env=ENV))

In [10]:
# new_response = request_options_chains(session, "SPXW")
# options_chains_df = pd.DataFrame(new_response.json()["data"]["items"])

In [11]:
# columns = [
#     "underlying-symbol",
#     "option-type",
#     "strike-price",
#     "streamer-symbol",
#     "symbol",
# ]

# rows = (options_chains_df["days-to-expiration"] == 0) & (
#     options_chains_df["strike-price"].str.contains(r"588|590|591", regex=True)
# )

# options_chains_df.loc[rows, columns]

In [1]:
# session.close_session()